In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [18]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
white_wine_data_df = pd.read_csv(
    "Resources/winequality-white.csv")

def convert_to_binary(value):
    if value >= 6:
        return 1
    else:
        return 0

white_wine_data_df['quality'] = white_wine_data_df['quality'].apply(convert_to_binary)

# Review the DataFrame
white_wine_data_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [19]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = white_wine_data_df["quality"]

# Separate the X variable, the features
x = white_wine_data_df.drop(columns="quality")

In [20]:
# Review the y variable Series
y.head()

0    1
1    1
2    1
3    1
4    1
Name: quality, dtype: int64

In [21]:
# Review the X variable DataFrame
x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [22]:
# Check the balance of our target values
y.value_counts()

1    3258
0    1640
Name: quality, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [23]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(x,
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [24]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver = 'lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

C:\Users\nickw\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [25]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).value_counts()

Prediction  Actual
1           1         701
            0         213
0           0         197
            1         114
dtype: int64

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [26]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.6703052521322759

In [27]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[197, 213],
       [114, 701]], dtype=int64)

In [29]:
# Print the classification report for the model
target_names = ["Bad Wine 0", "Good Wine 1"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

  Bad Wine 0       0.63      0.48      0.55       410
 Good Wine 1       0.77      0.86      0.81       815

    accuracy                           0.73      1225
   macro avg       0.70      0.67      0.68      1225
weighted avg       0.72      0.73      0.72      1225



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** It does very well, especially at determining healthy loans where it has a accuracy of 99%. The value of 1.00 precision for healthy loans compared to a value of 0.87 precision for high-risk loans indicates that while the model is extremely good at correctly marking healthy loans, it is less good at correctly marking unhealthy loans, but only marginally so. In plain language this means that the model is biased towards marking a high risk loan as healthy, however overall the model is quite good at correctly identifying the loans quality of the loans.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [15]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = ros.fit_resample(x, y)

In [22]:
# Count the distinct values of the resampled labels data
y_resampled_df = pd.Series(y_resampled)
y_resampled_df.value_counts()

0    75036
1    75036
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled,random_state=1)
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_resampled = LogisticRegression(solver = 'lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier_resampled.fit(X_train, y_train)

# Make a prediction using the testing data
predictions_resampled = classifier_resampled.predict(X_test)

pd.DataFrame({"Prediction": predictions_resampled, "Actual": y_test}).value_counts()

Prediction  Actual
0           0         18810
1           1         18521
            0            95
0           1            92
dtype: int64

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [26]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, predictions_resampled)

0.9950160462226244

In [27]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions_resampled)

array([[18810,    95],
       [   92, 18521]], dtype=int64)

In [28]:
# Print the classification report for the model
target_names = ["Healthy Loan 0", "High Risk Loan 1"]
print(classification_report(y_test, predictions_resampled, target_names=target_names))

                  precision    recall  f1-score   support

  Healthy Loan 0       1.00      0.99      1.00     18905
High Risk Loan 1       0.99      1.00      0.99     18613

        accuracy                           1.00     37518
       macro avg       1.00      1.00      1.00     37518
    weighted avg       1.00      1.00      1.00     37518



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** This model is superior to the previous one, by using random oversampling to address the problem of having fewer "High risk loans" in the data set to train and test the model on. This random oversampling model has a balanced accuracy score of 99.5%! It is able to predict both healthy and high risk loans to an equal level. Though it appears that this model out performs the previous model it is still important to consider the drawbacks of random oversampling. It is possible that the new model will not perform as well as the original model when introduced to new data, it may be that the model is overtrained to work well with only that data included in this training set. To investigate this hypothesis one should compare the accuracy of the models on new loan data.